In [10]:
## Create the wikipedia dataset
## changes have been made to WikiExtractor.py to be suitable for our requirements (no sub level paragraphs, one article per file)
%run -i WikiExtractor.py path_to_wiki_dump.bz2 -o .//etracted --min_text_length 2048 --sections --sections_top_level --filter_disambig_pages --processes 8 --bytes 2k

ERROR:root:args.bytes[-1]: 2k


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_wiki_dump.bz2'

In [55]:
import re

class TextParser(object):
        def __init__ (self, id_string="id",category_string="categories",title_string="title",**kwargs):
            EQUAL_GROUP = r'\=\"(.*?)\"'
            self.id_string= "{}{}".format(id_string,EQUAL_GROUP)
            self.category_string= "{}{}".format(category_string,EQUAL_GROUP)
            self.title_string= "{}{}".format(title_string,EQUAL_GROUP)
        
        def extract_title_id_categories(self,path,attribiutes_line):
            id_search = re.search(self.id_string, attribiutes_line, re.IGNORECASE)
            if id_search:
                id = id_search.group(1)

            title_search = re.search(self.title_string, attribiutes_line, re.IGNORECASE)
            if title_search:
                title = title_search.group(1)

            category_search = re.search(self.category_string, attribiutes_line, re.IGNORECASE)
            if category_search:
                categories = category_search.group(1).split(", ")

            try:
                return id,title,categories
            except:
                print("one of {}|{}|{} was missing in path {}".format(self.id_string,self.category_string,self.title_string,path))

        def __call__(self,path):
            # Using readlines() 
            article = open(path, 'r', encoding='utf-8') 
            lines = article.readlines() 
            if(len(lines) < 2):
                print("File: {} is empty".format(path))
                return -1,0,0

            attribiutes_line = lines.pop(0) # first line is attribiutes
            id,title,categories = self.extract_title_id_categories(path,attribiutes_line)
            article_name = lines.pop(0) 

            ##TODO split paragraph-name tuples
            return id,title,categories
            

        

In [58]:
import glob
import json


##TODO############
##Make sure no more that one doc in page
##add wget of wiki dump
##Download in server

####################

## Create title-doc_id dict and doc_id-[path] dict
PATH_TO_PARSED_WIKI = "C:\\Users\\t-dvginz\\PycharmProjects\\docBert\\data\\wikiextractor\\test_folder"

regex = PATH_TO_PARSED_WIKI + '\\**\\*[0-9][0-9]'

## Need running index for data loader
text_parser = TextParser()
name_to_running_id_dict = {}
running_id_to_path_title_categories = {}
idx=0
for idx_parse,wiki_f in enumerate(glob.iglob(regex, recursive=True)):      #iglob because too big for memory
    id,title,categories = text_parser(wiki_f)
    if(id==-1):continue
    name_to_running_id_dict[title] = idx
    running_id_to_path_title_categories[idx] = {"title":title,"path":wiki_f,"categories":categories,"id":id}
    
    if idx > 5: break
    idx +=1

    if(idx_parse % 10000 == 0): print("parsed {} articles".format(idx_parse))

print(json.dumps(name_to_running_id_dict,sort_keys=True, indent=4))
print(json.dumps(running_id_to_path_title_categories,sort_keys=True, indent=4))

File: C:\Users\t-dvginz\PycharmProjects\docBert\data\wikiextractor\test_folder\AA\wiki_00 is empty
{
    "A": 4,
    "Achilles": 6,
    "Alabama": 5,
    "Albedo": 3,
    "Anarchism": 1,
    "Autism": 2
}
{
    "1": {
        "categories": [
            "Anti-fascism",
            "Anti-capitalism",
            "Political ideologies",
            "Political culture",
            "Far-left politics",
            "Anarchism",
            "Political movements",
            "Economic ideologies",
            "Libertarianism",
            "Libertarian socialism"
        ],
        "id": "12",
        "path": "C:\\Users\\t-dvginz\\PycharmProjects\\docBert\\data\\wikiextractor\\test_folder\\AA\\wiki_01",
        "title": "Anarchism"
    },
    "2": {
        "categories": [
            "Neurological disorders in children",
            "Autism",
            "Pervasive developmental disorders",
            "Psychiatric diagnosis",
            "Neurological disorders",
            "Communication